In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [57]:
def load_data():
    
    # load panda pkl files
    data = pd.read_pickle('data.pkl')
    nomination_onehot = pd.read_pickle('nomination_onehot.pkl')
    selected_performers_onehot = pd.read_pickle('selected_performers_onehot.pkl')
    selected_directors_onehot = pd.read_pickle('selected_directors_onehot.pkl')
    selected_studio_onehot = pd.read_pickle('selected_studio_onehot.pkl')
    selected_scriptwriter_onehot = pd.read_pickle('selected_scriptwriter_onehot.pkl')
    
    base_path = "predict_target/target_"
    target_data = pd.read_pickle('predict_target/target.pkl')
    target_nomination_onehot = pd.read_pickle(base_path + 'nomination_onehot.pkl')
    target_selected_performers_onehot = pd.read_pickle(base_path + 'selected_performers_onehot.pkl')
    target_selected_directors_onehot = pd.read_pickle(base_path + 'selected_directors_onehot.pkl')
    target_selected_studio_onehot = pd.read_pickle(base_path + 'selected_studio_onehot.pkl')
    target_selected_scriptwriter_onehot = pd.read_pickle(base_path + 'selected_scriptwriter_onehot.pkl')
    # review_dataframe = pd.read_pickle('review_dataframe.pkl')
    # tfidf = pd.read_pickle('tfidf015_025.pkl')
    
    data_len = 211
    target_data.index += data_len
    target_nomination_onehot.index += data_len
    target_selected_performers_onehot.index += data_len
    target_selected_directors_onehot.index += data_len
    target_selected_studio_onehot.index += data_len
    target_selected_scriptwriter_onehot.index += data_len
    
    # concat target files and others
    data = pd.concat([data, target_data])
    nomination_onehot = pd.concat([nomination_onehot, target_nomination_onehot])
    selected_performers_onehot = pd.concat([selected_performers_onehot, target_selected_performers_onehot])
    selected_directors_onehot = pd.concat([selected_directors_onehot, target_selected_directors_onehot])
    selected_studio_onehot = pd.concat([selected_studio_onehot, target_selected_studio_onehot])
    selected_scriptwriter_onehotp = pd.concat([selected_scriptwriter_onehot, target_selected_scriptwriter_onehot])
    
    # selected_directors_onehotとselected_scriptwriter_onehotの重複した人
    duplicate_scriptwriter = set(selected_directors_onehot.columns) & set(selected_scriptwriter_onehot.columns)
    selected_scriptwriter_onehot = selected_scriptwriter_onehot.drop(duplicate_scriptwriter, axis=1)
          
    df = pd.concat(
        [
            nomination_onehot, 
            selected_performers_onehot,
            selected_directors_onehot,
            selected_studio_onehot,
            selected_scriptwriter_onehot,
            data["screen_time"],
            #tfidf
        ],
        axis=1,
        sort=False,
    )
    # 共線性の高いカラムを除く
    drop_clm = ['吉田一夫']
    df = df.drop(drop_clm, axis=1)
    
    # 取得できなかった上映時間(screen_time == -1)を平均で埋める
        # df[df["screen_time"] == -1] = df.mean().screen_time <- 良くない例
    df["screen_time"] = df["screen_time"].replace(-1, df["screen_time"].mean())
    
    # データセットとして扱うのに必要なyear, prizeのフラグを付与する
    df = pd.concat(
        [df, data["year"], data["prize"]], axis=1
    )
    
    df.fillna(0, inplace=True)
    return df

In [59]:
df = load_data()
df.head()

/home/kondo/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,blue_ribbon_award,golden_gross,hochi_eigashou,kinejun_best_ten,mainichi_film_award,nikkan_sports,中井貴一,丹波哲郎,井川比佐志,仲代達矢,...,澤井信一郎,田中陽造,神波史男,筒井ともみ,荒井晴彦,那須真知子,鄭義信,screen_time,year,prize
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108.0,1978,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,161.0,1978,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.0,1978,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,169.0,1978,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,1978,0


In [61]:
def standard_scale(year):
    scaler = StandardScaler()
    
    x_columns = df.drop(["year", "prize"], axis=1).columns
    
    train_x = df[df["year"] != year].drop(["year", "prize"], axis=1).values
    test_x = df[df["year"] == year].drop(["year", "prize"], axis=1).values
    train_y_df = df[df["year"] != year]["prize"]
    test_y_df = df[df["year"] == year]["prize"]
    
    scaler.fit(train_x)
    std_train_x = scaler.transform(train_x)
    std_test_x = scaler.transform(test_x)
    
    std_train_x_df = pd.DataFrame(std_train_x, columns=x_columns)
    std_test_x_df = pd.DataFrame(std_test_x, columns=x_columns)
    
    # インデックスの調整
    std_train_x_df.index.name = 'id'
    std_test_x_df.index.name = 'id'
    std_train_x_df.index += 1
    std_test_x_df.index += 1
    
    # pickleで保存
    base_path = "../std_data/"
    std_train_x_df.to_pickle(base_path  + "train/{}_x.pkl".format(str(year)))
    std_test_x_df.to_pickle(base_path + "test/{}_x.pkl".format(str(year)))
    train_y_df.to_pickle(base_path + "train/{}_y.pkl".format(str(year)))
    test_y_df.to_pickle(base_path + "test/{}_y.pkl".format(str(year)))
    return std_train_x_df, std_test_x_df, train_y_df, test_y_df

std_train_x_df, std_test_x_df, train_y_df, test_y_df = standard_scale(2020)

In [63]:
std_test_x_df

,blue_ribbon_award,golden_gross,hochi_eigashou,kinejun_best_ten,mainichi_film_award,nikkan_sports,中井貴一,丹波哲郎,井川比佐志,仲代達矢,...,李相日,松田寛夫,澤井信一郎,田中陽造,神波史男,筒井ともみ,荒井晴彦,那須真知子,鄭義信,screen_time
id,,,,,,,,,,,,,,,,,,,,,
1,-0.358249,-0.198517,-0.391159,-0.374887,-0.383065,-0.366618,-0.256235,-0.245564,-0.286446,-0.234521,...,-0.120096,-0.155794,-0.120096,-0.120096,-0.120096,-0.13901,-0.120096,-0.120096,-0.120096,0.269963
2,-0.358249,-0.198517,-0.391159,-0.374887,-0.383065,-0.366618,-0.256235,-0.245564,-0.286446,-0.234521,...,-0.120096,-0.155794,-0.120096,-0.120096,-0.120096,-0.13901,-0.120096,-0.120096,-0.120096,0.461399
3,-0.358249,-0.198517,-0.391159,-0.374887,-0.383065,-0.366618,-0.256235,-0.245564,-0.286446,-0.234521,...,-0.120096,-0.155794,-0.120096,-0.120096,-0.120096,-0.13901,-0.120096,-0.120096,-0.120096,0.525211
4,-0.358249,-0.198517,-0.391159,-0.374887,-0.383065,-0.366618,-0.256235,-0.245564,-0.286446,-0.234521,...,-0.120096,-0.155794,-0.120096,-0.120096,-0.120096,-0.13901,-0.120096,-0.120096,-0.120096,-0.431971
5,-0.358249,-0.198517,-0.391159,-0.374887,-0.383065,-0.366618,-0.256235,-0.245564,-0.286446,-0.234521,...,-0.120096,-0.155794,-0.120096,-0.120096,-0.120096,-0.13901,-0.120096,-0.120096,-0.120096,-1.197717


In [42]:
data = pd.read_pickle('data.pkl')
target_data = pd.read_pickle('predict_target/target.pkl')
target_data.index += 211

data = pd.concat([data, target_data], axis=0, sort=True)
data

,director,other_nominates,performers,prize,production_studio,screen_time,scriptwriter,selected_directors,selected_performers,selected_scriptwriter,selected_studio,title,year
id,,,,,,,,,,,,,
1,[山田洋次],"[hochi_eigashou, blue_ribbon_award]","[高倉健, 倍賞千恵子, 桃井かおり, 武田鉄矢, 渥美清]",1,[松竹],108,"[山田洋次, 朝間義隆]",[山田洋次],[高倉健],"[山田洋次, 朝間義隆]",[松竹],幸福の黄色いハンカチ,1978
2,"[浦山桐郎, 橋本幸治]",[],"[田中健, 大竹しのぶ, いしだあゆみ, 高橋悦史, 伊東辰夫, 高瀬春奈, 高山彰, 梅宮...",0,[東宝],161,"[早坂暁, 浦山桐郎]",[],[大竹しのぶ],[早坂暁],[東宝],青春の門 自立篇（1977）,1978
3,"[新藤兼人, 小松崎和男]",[],"[高橋竹山, 林隆三, 乙羽信子, 金井大, 倍賞美津子, 島村佳江, 観世栄夫, 根岸明美...",0,[独立映画センター],125,[新藤兼人],[],[倍賞美津子],[新藤兼人],[],竹山ひとり旅,1978
4,"[森谷司郎, 神山征二郎]",[],"[島田正吾, 大滝秀治, 高倉健, 丹波哲郎, 藤岡琢也, 浜田晃, 加藤健一, 江幡連, ...",0,[東宝],169,[橋本忍],[神山征二郎],"[大滝秀治, 緒形拳, 神山繁, 高倉健, 丹波哲郎]",[],[東宝],八甲田山,1978
5,"[篠田正浩, 永井正夫]",[],"[岩下志麻, 原田芳雄, 奈良岡朋子, 神保共子, 横山リエ, 宮沢亜古, 中村恵子, 殿山...",0,[東宝],117,"[長谷部慶治, 篠田正浩]",[篠田正浩],"[西田敏行, 樹木希林, 浜村純]",[],[東宝],はなれ瞽女おりん,1978
6,"[野村芳太郎, 大嶺俊順]",[mainichi_film_award],"[永島敏行, 松坂慶子, 大竹しのぶ, 佐分利信, 中野誠也, 磯部勉, 芦田伸介, 丹波哲...",1,[松竹],138,[新藤兼人],[],"[永島敏行, 丹波哲郎, 大竹しのぶ]",[新藤兼人],[松竹],事件,1979
7,[大島渚],[],"[長谷川真砂美, 伊佐山ひろ子, 佐藤慶, 殿山泰司, 田村高廣, 吉行和子, 藤竜也, 杉...",0,[],108,[],[],"[田村高廣, 吉行和子]",[],[],愛の亡霊,1979
8,"[野村芳太郎, 松原信吾]",[],"[岩下志麻, 緒形拳, 岩瀬浩規, 吉沢美幸, 石井旬, 蟹江敬三, 穂積隆信, 大滝秀治,...",0,[松竹],110,[井手雅人],[],"[大滝秀治, 緒形拳, 大竹しのぶ, 浜村純, 田中邦衛]",[],[松竹],鬼畜,1979
9,"[東陽一, 青池憲司]","[hochi_eigashou, blue_ribbon_award, kinejun_be...","[永島敏行, 吉田次昭, 森下愛子, 志方亜紀子, 島倉千代子, 内藤武敏, 峰岸徹, 片桐...",0,[ATG],103,[寺山修司],[],[永島敏行],[],[],サード,1979


id
1      1978
2      1978
3      1978
4      1978
5      1978
       ... 
207    2019
208    2019
209    2019
210    2019
211    2019
Name: year, Length: 211, dtype: int64

In [69]:
unko = pd.read_pickle("../../data/dataframes/std_data/train/1978_y.pkl")
unko

id
6      1
7      0
8      0
9      0
10     0
      ..
207    1
208    0
209    0
210    0
211    0
Name: prize, Length: 206, dtype: int64